In [1]:
from robot_descriptions.a1_description import URDF_PATH
from darli.backend import PinocchioBackend, CasadiBackend, JointType
from darli.modeling import Robot, Functional
from darli.modeling.integrators import ForwardEuler, RK4, MidPoint
import numpy as np

In [2]:
dt = 1e-3
n_steps = 20
control_sampling = 5e-3
samples = int(dt * n_steps / control_sampling)

In [3]:
model = Functional(CasadiBackend(URDF_PATH, root_joint=JointType.FREE_FLYER))
model.update_selector(passive_joints=range(6))

In [4]:
np.random.seed(0)
tau = np.random.randn(model.nu)

tau

array([ 1.76405235,  0.40015721,  0.97873798,  2.2408932 ,  1.86755799,
       -0.97727788,  0.95008842, -0.15135721, -0.10321885,  0.4105985 ,
        0.14404357,  1.45427351])

In [5]:
q0 = np.array([0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
v0 = np.zeros(model.nv)

x0 = np.concatenate([q0, v0])

x0.shape, model.nq, model.nv

((37,), 19, 18)

## Functional way

In [6]:
model.state_space.integrator

In [7]:
euler_roll = model.state_space.rollout(
    dt=dt, n_steps=n_steps, control_sampling=control_sampling
)

euler_roll

Function(rollout:(state[37],controls[12x4])->(next_state[37]) SXFunction)

In [8]:
%timeit euler_roll()

554 µs ± 6.98 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [9]:
model.state_space.set_integrator(MidPoint)

midpoint_roll = model.state_space.rollout(
    dt=dt, n_steps=n_steps, control_sampling=control_sampling
)

midpoint_roll

Function(rollout:(state[37],controls[12x4])->(next_state[37]) SXFunction)

In [10]:
%timeit midpoint_roll()

1.1 ms ± 12.9 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [11]:
model.state_space.set_integrator(RK4)

rk4_roll = model.state_space.rollout(
    dt=dt, n_steps=n_steps, control_sampling=control_sampling
)

rk4_roll

Function(rollout:(state[37],controls[12x4])->(next_state[37]) SXFunction)

In [12]:
%timeit rk4_roll()

2.2 ms ± 34.6 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


## Pinocchio way

In [13]:
model = Robot(PinocchioBackend(URDF_PATH, root_joint=JointType.FREE_FLYER))
model.update_selector(passive_joints=range(6))

In [14]:
q0 = np.array([0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
v0 = np.zeros(model.nv)

x0 = np.concatenate([q0, v0])

np.random.seed(0)
tau = np.random.randn(model.nu, samples) * 0

In [15]:
model.state_space.integrator

In [16]:
%timeit model.state_space.rollout(x0, tau, dt=dt, n_steps=n_steps, control_sampling=control_sampling)

181 µs ± 3.23 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [17]:
euler_state = model.state_space.rollout(
    x0, tau, dt=dt, n_steps=n_steps, control_sampling=control_sampling
)

euler_state, "quaternion norm", np.linalg.norm(euler_state[3:7])

(array([ 0.       ,  0.       , -0.0018639,  0.       ,  0.       ,
         0.       ,  1.       ,  0.       ,  0.       ,  0.       ,
         0.       ,  0.       ,  0.       ,  0.       ,  0.       ,
         0.       ,  0.       ,  0.       ,  0.       ,  0.       ,
         0.       , -0.1962   ,  0.       ,  0.       ,  0.       ,
         0.       ,  0.       ,  0.       ,  0.       ,  0.       ,
         0.       ,  0.       ,  0.       ,  0.       ,  0.       ,
         0.       ,  0.       ]),
 'quaternion norm',
 1.0)

In [18]:
model.state_space.set_integrator(MidPoint)

%timeit model.state_space.rollout(x0, tau, dt=dt, n_steps=n_steps, control_sampling=control_sampling)

366 µs ± 20.1 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [19]:
midpoint_state = model.state_space.rollout(
    x0, tau, dt=dt, n_steps=n_steps, control_sampling=control_sampling
)

midpoint_state, "quaternion norm", np.linalg.norm(midpoint_state[3:7])

(array([ 0.        ,  0.        , -0.00415208,  0.        ,  0.        ,
         0.        ,  1.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        , -0.289395  ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ]),
 'quaternion norm',
 1.0)

In [20]:
model.state_space.set_integrator(RK4)

%timeit model.state_space.rollout(x0, tau, dt=dt, n_steps=n_steps, control_sampling=control_sampling)

745 µs ± 8.92 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [21]:
rk4_state = model.state_space.rollout(
    x0, tau, dt=dt, n_steps=n_steps, control_sampling=control_sampling
)

rk4_state, "quaternion norm", np.linalg.norm(rk4_state[3:7])

(array([ 0.        ,  0.        , -0.01604998,  0.        ,  0.        ,
         0.        ,  1.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        , -0.56898   ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ]),
 'quaternion norm',
 1.0)